*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit

# Input generator code

Input generator code creates inp.xml input file from a simplified input file. To use this code via AiiDA, it is needed to implement a corresponding python class that contaings all the logic needed for the use of this code. The class `FleurinputgenCalculation` and it's parser already exist in AiiDA-FLEUR plugin. In this tutorial we will learn how to use it in AiiDA inteface.

First things first - let us get familiar with inputs of the FleurinputgenCalculation.

## Inputs of the input calculation

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Inpgen code| yes |
|structure | StructureData | Structure data node| yes |
|parameters | Dict | FLAPW parameters| no |
|settings | Dict | special settings | no |

**code** is a code object representing inpgen code and stored in the database. It is also required to provide input **structure**. Finally, optional **parameters** are a dictionary of parameters that will be written into the simplified input file. For instance, if one wants to generate a line

    &kpt
      div1=24   div2=20   div3=1   tkb=0.001 /

in the simplified input file, it is necessary to append

    'kpt': {
        'div1': 24,
        'div2' : 20,
        'div3' : 1
        }

to the **parameters** dictionary. 

**settings** is used to modify a list of files to be copied to/from/between calculation folders. We will not cover this in the tutorial.


## Inpgen code submission

First of all, we need to import `FleurinputgenCalculation` class.

In [ ]:
from aiida_fleur.calculation.inpgen import FleurinputgenCalculation

and prepare input nodes. Configured and ready-to-use inpgen code object is already stored in your database under the number pk=xxx. Now you can simple load it from the database:

In [ ]:
code = load_node(xxx)

We are going to use a structure that you have created in tutorial 1. Please, find a pk of an Fe monolayer structure and load in via the same command:

In [ ]:
structure = load_node(xxx)

Finally, we need to setup a **parameters** dictionary. For now we are going to set k-mesh and kmax only:

In [ ]:
parameters = Dict(dict={
    'comp': {
        'kmax': 3.85,
        },
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

In [ ]:
res = submit(FleurinputgenCalculation, code=code, structure=structure, parameters=parameters)

The calculation has to finish fast - you will probably not be able to catch it in not 'FINISHED' state.
You can explore it's outputs: